In [202]:
-- wyswietlenie baz danych na serwerze
select [name] as database_name,
    database_id,
    create_date
from sys.databases
order by name

(2 rows affected)

Total execution time: 00:00:00.030

database_name,database_id,create_date
master,1,2020-12-07 18:39:43.607
okno-bd-projekt,5,2020-12-07 19:24:48.477


In [203]:
-- DROP WSZYSTKICH KLUCZY
while(exists(select 1 from INFORMATION_SCHEMA.TABLE_CONSTRAINTS where CONSTRAINT_TYPE='FOREIGN KEY'))
begin
 declare @sql nvarchar(2000)
 SELECT TOP 1 @sql=('ALTER TABLE ' + TABLE_SCHEMA + '.[' + TABLE_NAME
 + '] DROP CONSTRAINT [' + CONSTRAINT_NAME + ']')
 FROM information_schema.table_constraints
 WHERE CONSTRAINT_TYPE = 'FOREIGN KEY'
 exec (@sql)
 --PRINT @sql
end


Commands completed successfully.

Total execution time: 00:00:00.197

In [204]:
-- DROP WSZYSTKICH TABEL
while(exists(select 1 from INFORMATION_SCHEMA.TABLES 
             where TABLE_NAME != '__MigrationHistory' 
             AND TABLE_TYPE = 'BASE TABLE'))
begin
 declare @sql nvarchar(2000)
 SELECT TOP 1 @sql=('DROP TABLE ' + TABLE_SCHEMA + '.[' + TABLE_NAME
 + ']')
 FROM INFORMATION_SCHEMA.TABLES
 WHERE TABLE_NAME != '__MigrationHistory' AND TABLE_TYPE = 'BASE TABLE'
exec (@sql)
 /* you dont need this line, it just shows what was executed */
 --PRINT @sql
end

Commands completed successfully.

Total execution time: 00:00:00.148

In [205]:
-- DROP FUNCKJI
DROP FUNCTION IF EXISTS [dbo].[IloscAutorowPracy]
GO
DROP FUNCTION IF EXISTS [dbo].[CzyMozeBycPromotorem]
GO


Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.064

Tworzenie tabel

In [206]:
-- Stworzenie tabel
CREATE TABLE TypStudiow(
    ID_TypStudiow SMALLINT IDENTITY PRIMARY KEY,
    NazwaTypu VARCHAR(20) NOT NULL
)

CREATE TABLE Studia(
	ID_Studia INT IDENTITY PRIMARY KEY,
	Wydzial varchar(max) NOT NULL,
	Kierunek varchar(max) NOT NULL
)


CREATE TABLE StopienNaukowy(
	ID_StopienNaukowy INT IDENTITY PRIMARY KEY,
	Stopien varchar(20) NOT NULL
)

CREATE TABLE Slowo(
	ID_Slowo INT IDENTITY PRIMARY KEY,
	Slowo varchar(50) NOT NULL
)

CREATE TABLE MiejsceObrony(
	ID_MiejsceObrony INT IDENTITY PRIMARY KEY,
	Budynek VARCHAR(max) NOT NULL,
	Adres VARCHAR(50) NOT NULL,
	Sala SMALLINT NOT NULL
)

CREATE TABLE Pracownik(
	ID_Pracownik INT IDENTITY PRIMARY KEY,
	ID_StopienNaukowy INT FOREIGN KEY REFERENCES StopienNaukowy(ID_StopienNaukowy),
	Imie varchar(100) NOT NULL,
	Nazwisko varchar(100) NOT NULL,
	Telefon INT,
	Email varchar(100)
)

CREATE TABLE Student(
	ID_Student BIGINT PRIMARY KEY CHECK(ID_Student <10000000000),
	Imie varchar(100) NOT NULL,
	Nazwisko varchar(100) NOT NULL,
	Telefon INT,
	Email varchar(50)
)

CREATE TABLE StudiowanyKierunek(
	ID_Student BIGINT FOREIGN KEY REFERENCES Student(ID_Student),
	ID_Studia INT FOREIGN KEY REFERENCES Studia(ID_Studia),
	PRIMARY KEY(ID_Student,ID_Studia)
)

CREATE TABLE PracaDyplomowa(
	ID_Praca INT IDENTITY PRIMARY KEY,
	TematPracy varchar(max) NOT NULL,
	ID_TypStudiow SMALLINT NOT NULL ,
	ID_Promotor INT FOREIGN KEY REFERENCES Pracownik(ID_Pracownik),
	OcenaPromotora DEC(2,1) CHECK(OcenaPromotora = 2 OR OcenaPromotora = 2.5 OR OcenaPromotora = 3 OR OcenaPromotora = 3.5 OR OcenaPromotora = 4 OR OcenaPromotora = 4.5 OR OcenaPromotora = 5),
	DataZlozenia DATETIME,
	FOREIGN KEY (ID_TypStudiow) REFERENCES TypStudiow(ID_TypStudiow)
)

CREATE TABLE Autor(
	ID_Praca INt FOREIGN KEY REFERENCES PracaDyplomowa(ID_Praca),
	ID_Student BIGINT FOREIGN KEY REFERENCES Student(ID_Student)
	PRIMARY KEY(ID_Praca,ID_Student)
)

CREATE TABLE Obrona(
	ID_Obrona INT IDENTITY PRIMARY KEY,
	ID_Przewodniczacy INT FOREIGN KEY REFERENCES Pracownik(ID_Pracownik) NOT NULL,
	ID_Praca INT NOT NULL,
	OcenaKoncowa DEC(2,1) CHECK(OcenaKoncowa = 2 OR OcenaKoncowa = 2.5 OR OcenaKoncowa = 3 OR OcenaKoncowa = 3.5 OR OcenaKoncowa = 4 OR OcenaKoncowa = 4.5 OR OcenaKoncowa = 5),
	DataObrony DATETIME,
	ID_MiejsceObrony INT FOREIGN KEY REFERENCES MiejsceObrony(ID_MiejsceObrony) 
)

CREATE TABLE SlowaKluczowe(
	ID_Slowo INT FOREIGN KEY REFERENCES Slowo(ID_Slowo),
	ID_Praca INT FOREIGN KEY REFERENCES PracaDyplomowa(ID_Praca),
	PRIMARY KEY(ID_Slowo,ID_Praca)
)


CREATE TABLE Recenzja(
	ID_Praca INT FOREIGN KEY REFERENCES PracaDyplomowa(ID_Praca),
	ID_Recenzent INT FOREIGN KEY REFERENCES Pracownik(ID_Pracownik),
	Ocena DEC(2,1) CHECK(Ocena = 2 OR Ocena = 2.5 OR Ocena = 3 OR Ocena = 3.5 OR Ocena = 4 OR Ocena = 4.5 OR Ocena = 5),
	DataOceny DATETIME,
	Komentarz varchar(max)
	PRIMARY KEY(ID_Praca,ID_Recenzent)
)

Commands completed successfully.

Total execution time: 00:00:00.131

In [207]:
--FUNKCJA LICZACA ILU JEST AUTOROW PRACY DYPLOMOWEJ AKTUALNIE
CREATE FUNCTION IloscAutorowPracy(@PRACA INT)
RETURNS INT
BEGIN
	DECLARE @return INT;

	SELECT @return = COUNT(*)
	FROM Autor
	WHERE ID_Praca = @PRACA

	RETURN @return
END;

Commands completed successfully.

Total execution time: 00:00:00.033

In [208]:
ALTER TABLE Autor ADD CHECK(dbo.IloscAutorowPracy(ID_Praca)<=3)

Commands completed successfully.

Total execution time: 00:00:00.031

In [209]:
--CZy moze byc promotorem
CREATE FUNCTION CzyMozeBycPromotorem(@idpracownika INT)
RETURNS BIT
BEGIN
    DECLARE @stiopien INT;
    SET @stiopien = NULL;
    DECLARE @return BIT;
    
    SELECT @stiopien = ID_StopienNaukowy 
    FROM Pracownik 
    WHERE ID_Pracownik = @idpracownika;
   
    IF(@stiopien IS NOT NULL)
        BEGIN
        SET @return =  CAST (1 AS BIT)
        END
    ELSE
        BEGIN
        SET @return = CAST(0 AS BIT)
        END
    RETURN @return
END;

Commands completed successfully.

Total execution time: 00:00:00.034

In [210]:
ALTER TABLE PracaDyplomowa ADD CHECK(dbo.CzyMozeBycPromotorem(ID_Promotor)=CAST(1 AS BIT))

Commands completed successfully.

Total execution time: 00:00:00.036

# Uzupełnianie Tabel

In [211]:
-- TYP STUDIOW
INSERT INTO TypStudiow(NazwaTypu) VALUES ('licencjackie');
INSERT INTO TypStudiow(NazwaTypu) VALUES ('inzynierskie');
INSERT INTO TypStudiow(NazwaTypu) VALUES ('magisterskie');

--INSERT STUDIA
INSERT INTO Studia(Wydzial,Kierunek) VALUES ('Mechatroniki','Automatyka, Robotyka i Informatyka Przemysłowa')
INSERT INTO Studia(Wydzial,Kierunek) VALUES ('Mechatroniki','Mechatronika')
INSERT INTO Studia(Wydzial,Kierunek) VALUES ('Elektryczny','Iformatyka stosowana')

--INSERT STOPNIE NAUKOWE
INSERT INTO StopienNaukowy(Stopien) VALUES ('Doktor')
INSERT INTO StopienNaukowy(Stopien) VALUES ('Profesor')

------------------------------------------------------------------------------------------------------------------------------
INSERT INTO Slowo(Slowo) VALUES ('Sztuczna inteligancja')
INSERT INTO Slowo(Slowo) VALUES ('Kwiat')
INSERT INTO Slowo(Slowo) VALUES ('Kot')

INSERT INTO MiejsceObrony(Budynek,Adres,Sala) VALUES ('Wydział Matematyki i Nauk Informacyjnych','Kwiatowa 1',1)

INSERT INTO Pracownik(Imie,Nazwisko,ID_StopienNaukowy,Telefon,Email) VALUES ('Olga', 'Olszewska',2,123456789,'mail@gmail.com')
INSERT INTO Pracownik(Imie,Nazwisko,ID_StopienNaukowy,Telefon,Email) VALUES ('Anna', 'Anielska',1,123456789,'mail@gmail.com')

INSERT INTO Student(ID_Student,Imie,Nazwisko,Telefon,Email) VALUES (297225,'Marcin','Bobiński',123456789,'marcin@gmail.com')
INSERT INTO Student(ID_Student,Imie,Nazwisko,Telefon,Email) VALUES (297226,'Dominik','Polak',123456789,'dominik@gmail.com')
INSERT INTO Student(ID_Student,Imie,Nazwisko,Telefon,Email) VALUES (297227,'Marcel','Grapiec',123456789,'marcel@gmail.com')
INSERT INTO Student(ID_Student,Imie,Nazwisko,Telefon,Email) VALUES (297228,'Magda','Kwiatkowska',123456789,'madzia@gmail.com')


INSERT INTO StudiowanyKierunek(ID_Student,ID_Studia) VALUES (297225,1)
INSERT INTO StudiowanyKierunek(ID_Student,ID_Studia) VALUES (297226,1)
INSERT INTO StudiowanyKierunek(ID_Student,ID_Studia) VALUES (297227,1)
INSERT INTO StudiowanyKierunek(ID_Student,ID_Studia) VALUES (297228,1)

INSERT INTO PracaDyplomowa(TematPracy,ID_TypStudiow,ID_Promotor,OcenaPromotora,DataZlozenia) VALUES ('Grzyby',2,1,3,GETDATE())
INSERT INTO PracaDyplomowa(TematPracy,ID_TypStudiow,ID_Promotor,OcenaPromotora,DataZlozenia) VALUES ('Mchy',1,1,3,NULL)
INSERT INTO PracaDyplomowa(TematPracy,ID_TypStudiow,ID_Promotor,OcenaPromotora,DataZlozenia) VALUES ('Kora',2,2,3,GETDATE())
INSERT INTO PracaDyplomowa(TematPracy,ID_TypStudiow,ID_Promotor,OcenaPromotora,DataZlozenia) VALUES ('Kotki',1,2,3,GETDATE())

INSERT INTO Autor(ID_Praca ,ID_Student) VALUES (1,297225)
INSERT INTO Autor(ID_Praca ,ID_Student) VALUES (1,297226)
INSERT INTO Autor(ID_Praca ,ID_Student) VALUES (1,297227)
INSERT INTO Autor(ID_Praca ,ID_Student) VALUES (2,297228)
INSERT INTO Autor(ID_Praca ,ID_Student) VALUES (3,297225)
INSERT INTO Autor(ID_Praca ,ID_Student) VALUES (4,297226)

INSERT INTO Obrona(ID_Przewodniczacy,ID_Praca,OcenaKoncowa,DataObrony,ID_MiejsceObrony) VALUES (1,1,3,NULL,1)
INSERT INTO Obrona(ID_Przewodniczacy,ID_Praca,OcenaKoncowa,DataObrony,ID_MiejsceObrony) VALUES (1,2,4,'2021-01-10 21:18:20.110',1)
INSERT INTO Obrona(ID_Przewodniczacy,ID_Praca,OcenaKoncowa,DataObrony,ID_MiejsceObrony) VALUES (2,3,5,'2020-01-10 21:18:20.110',1)
INSERT INTO Obrona(ID_Przewodniczacy,ID_Praca,OcenaKoncowa,DataObrony,ID_MiejsceObrony) VALUES (2,4,4.5,'2020-01-10 21:18:20.110',1)

INSERT INTO SlowaKluczowe(ID_Slowo,ID_Praca) VALUES (1,1)
INSERT INTO SlowaKluczowe(ID_Slowo,ID_Praca) VALUES (1,2)
INSERT INTO SlowaKluczowe(ID_Slowo,ID_Praca) VALUES (2,3)
INSERT INTO SlowaKluczowe(ID_Slowo,ID_Praca) VALUES (3,2)
INSERT INTO SlowaKluczowe(ID_Slowo,ID_Praca) VALUES (1,3)
INSERT INTO SlowaKluczowe(ID_Slowo,ID_Praca) VALUES (1,4)

INSERT INTO Recenzja(ID_Praca,ID_Recenzent,Ocena,DataOceny,Komentarz) VALUES (1,1,4,NULL,'Cały komentarz na temat pracy')

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

...

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.228

In [212]:
SELECT * FROM TypStudiow
SELECT * FROM Studia
SELECT * FROM StopienNaukowy
SELECT * FROM Slowo
SELECT * FROM MiejsceObrony
SELECT * FROM Pracownik
SELECT * FROM Student
SELECT * FROM StudiowanyKierunek
SELECT * FROM PracaDyplomowa
SELECT * FROM Autor
SELECT * FROM Obrona
SELECT * FROM SlowaKluczowe
SELECT * FROM Recenzja

(3 rows affected)

(3 rows affected)

(2 rows affected)

(3 rows affected)

(1 row affected)

...

ID_Praca,TematPracy,ID_TypStudiow,ID_Promotor,OcenaPromotora,DataZlozenia
1,Grzyby,2,1,"3,0",2021-01-10 23:06:04.960
2,Mchy,1,1,"3,0",NULL
3,Kora,2,2,"3,0",2021-01-10 23:06:04.977
4,Kotki,1,2,"3,0",2021-01-10 23:06:04.980


ID_Praca,ID_Student
1,297225
1,297226
1,297227
2,297228
3,297225
4,297226


ID_Obrona,ID_Przewodniczacy,ID_Praca,OcenaKoncowa,DataObrony,ID_MiejsceObrony
1,1,1,"3,0",NULL,1
2,1,2,"4,0",2021-01-10 21:18:20.110,1
3,2,3,"5,0",2020-01-10 21:18:20.110,1
4,2,4,"4,5",2020-01-10 21:18:20.110,1


ID_Slowo,ID_Praca
1,1
1,2
1,3
1,4
2,3
3,2


ID_Praca,ID_Recenzent,Ocena,DataOceny,Komentarz
1,1,"4,0",NULL,Caly komentarz na temat pracy


# ZAPYTANIA

In [213]:
--wyszukiwanie prac związanych z daną tematyką
SELECT PracaDyplomowa.ID_Praca,PracaDyplomowa.TematPracy,TypStudiow.NazwaTypu AS Rodzaj,AutorAGG.ImieNazwisko,AutorAGG.ID_Student,CONCAT(Pracownik.Imie,' ',Pracownik.Nazwisko) AS Promotor,
PracaDyplomowa.OcenaPromotora,PracaDyplomowa.DataZlozenia,Obrona.DataObrony,Obrona.OcenaKoncowa
FROM PracaDyplomowa 
LEFT JOIN SlowaKluczowe ON PracaDyplomowa.ID_Praca = SlowaKluczowe.ID_Praca
LEFT JOIN Slowo ON SlowaKluczowe.ID_Slowo = Slowo.ID_Slowo
LEFT JOIN TypStudiow ON PracaDyplomowa.ID_TypStudiow = TypStudiow.ID_TypStudiow
LEFT JOIN (
    SELECT Autor.ID_Praca AS ID_Praca,STRING_AGG(CONCAT(Student.Imie,' ',Student.Nazwisko),', ') AS ImieNazwisko,STRING_AGG(Student.ID_Student,', ') AS ID_Student
    FROM PracaDyplomowa
    LEFT JOIN Autor ON Autor.ID_Praca = PracaDyplomowa.ID_Praca
    LEFT JOIN Student ON Autor.ID_Student = Student.ID_Student
    GROUP BY Autor.ID_Praca
) AS AutorAGG ON AutorAGG.ID_Praca = PracaDyplomowa.ID_Praca
LEFT JOIN Obrona ON Obrona.ID_Praca = PracaDyplomowa.ID_Praca
LEFT JOIN Pracownik ON Obrona.ID_Przewodniczacy = Pracownik.ID_Pracownik
WHERE Slowo.Slowo = 'Sztuczna inteligancja'


(4 rows affected)

Total execution time: 00:00:00.100

ID_Praca,TematPracy,Rodzaj,ImieNazwisko,ID_Student,Promotor,OcenaPromotora,DataZlozenia,DataObrony,OcenaKoncowa
1,Grzyby,inzynierskie,"Marcin Bobinski, Dominik Grabiec, Marcel Polak","297225, 297226, 297227",Olga Olszewska,"3,0",2021-01-10 23:06:04.960,NULL,"3,0"
2,Mchy,licencjackie,Magda Nguyen Hong,297228,Olga Olszewska,"3,0",NULL,2021-01-10 21:18:20.110,"4,0"
3,Kora,inzynierskie,Marcin Bobinski,297225,Anna Anielska,"3,0",2021-01-10 23:06:04.977,2020-01-10 21:18:20.110,"5,0"
4,Kotki,licencjackie,Dominik Grabiec,297226,Anna Anielska,"3,0",2021-01-10 23:06:04.980,2020-01-10 21:18:20.110,"4,5"


In [214]:
--odnalezienie prac prowadzonych przez określonego promotora
SELECT PracaDyplomowa.ID_Praca,PracaDyplomowa.TematPracy,TypStudiow.NazwaTypu AS Rodzaj,AutorAGG.ImieNazwisko,AutorAGG.ID_Student,CONCAT(Pracownik.Imie,' ',Pracownik.Nazwisko) AS Promotor,
PracaDyplomowa.OcenaPromotora,PracaDyplomowa.DataZlozenia,Obrona.DataObrony,Obrona.OcenaKoncowa
FROM PracaDyplomowa 
LEFT JOIN Pracownik ON PracaDyplomowa.ID_Promotor = Pracownik.ID_Pracownik
LEFT JOIN TypStudiow ON PracaDyplomowa.ID_TypStudiow = TypStudiow.ID_TypStudiow
LEFT JOIN (
    SELECT Autor.ID_Praca AS ID_Praca,STRING_AGG(CONCAT(Student.Imie,' ',Student.Nazwisko),', ') AS ImieNazwisko,STRING_AGG(Student.ID_Student,', ') AS ID_Student
    FROM PracaDyplomowa
    LEFT JOIN Autor ON Autor.ID_Praca = PracaDyplomowa.ID_Praca
    LEFT JOIN Student ON Autor.ID_Student = Student.ID_Student
    GROUP BY Autor.ID_Praca
) AS AutorAGG ON AutorAGG.ID_Praca = PracaDyplomowa.ID_Praca
LEFT JOIN Obrona ON Obrona.ID_Praca = PracaDyplomowa.ID_Praca
WHERE Pracownik.ID_Pracownik = 1 -- LUB  Pracownik.Imie = 'Olga' AND Pracownik.Nazwisko = 'Olszewska'

(2 rows affected)

Total execution time: 00:00:00.054

ID_Praca,TematPracy,Rodzaj,ImieNazwisko,ID_Student,Promotor,OcenaPromotora,DataZlozenia,DataObrony,OcenaKoncowa
1,Grzyby,inzynierskie,"Marcin Bobinski, Dominik Grabiec, Marcel Polak","297225, 297226, 297227",Olga Olszewska,"3,0",2021-01-10 23:06:04.960,NULL,"3,0"
2,Mchy,licencjackie,Magda Nguyen Hong,297228,Olga Olszewska,"3,0",NULL,2021-01-10 21:18:20.110,"4,0"


In [215]:
--wyświetlenie informacji o pracy danego studenta
SELECT PracaDyplomowa.ID_Praca,PracaDyplomowa.TematPracy,TypStudiow.NazwaTypu AS Rodzaj,AutorAGG.ImieNazwisko,AutorAGG.ID_Student,CONCAT(Pracownik.Imie,' ',Pracownik.Nazwisko) AS Promotor,
PracaDyplomowa.OcenaPromotora,PracaDyplomowa.DataZlozenia,Obrona.DataObrony,Obrona.OcenaKoncowa
FROM PracaDyplomowa
LEFT JOIN Autor ON PracaDyplomowa.ID_Praca = Autor.ID_Praca
Left JOIN Student ON Autor.ID_Student = Student.ID_Student
LEFT JOIN Pracownik ON PracaDyplomowa.ID_Promotor = Pracownik.ID_Pracownik
LEFT JOIN TypStudiow ON PracaDyplomowa.ID_TypStudiow = TypStudiow.ID_TypStudiow
LEFT JOIN (
    SELECT Autor.ID_Praca AS ID_Praca,STRING_AGG(CONCAT(Student.Imie,' ',Student.Nazwisko),', ') AS ImieNazwisko,STRING_AGG(Student.ID_Student,', ') AS ID_Student
    FROM PracaDyplomowa
    LEFT JOIN Autor ON Autor.ID_Praca = PracaDyplomowa.ID_Praca
    LEFT JOIN Student ON Autor.ID_Student = Student.ID_Student
    GROUP BY Autor.ID_Praca
) AS AutorAGG ON AutorAGG.ID_Praca = PracaDyplomowa.ID_Praca
LEFT JOIN Obrona ON Obrona.ID_Praca = PracaDyplomowa.ID_Praca
WHERE Student.ID_Student = 297225

(2 rows affected)

Total execution time: 00:00:00.056

ID_Praca,TematPracy,Rodzaj,ImieNazwisko,ID_Student,Promotor,OcenaPromotora,DataZlozenia,DataObrony,OcenaKoncowa
1,Grzyby,inzynierskie,"Marcin Bobinski, Dominik Grabiec, Marcel Polak","297225, 297226, 297227",Olga Olszewska,"3,0",2021-01-10 23:06:04.960,NULL,"3,0"
3,Kora,inzynierskie,Marcin Bobinski,297225,Anna Anielska,"3,0",2021-01-10 23:06:04.977,2020-01-10 21:18:20.110,"5,0"


In [216]:
--wyświetlenie raportu o pracach obronionych na podanym kierunku studiów lub w podanym roku.
SELECT DISTINCT PracaDyplomowa.ID_Praca,PracaDyplomowa.TematPracy,TypStudiow.NazwaTypu AS Rodzaj,AutorAGG.ImieNazwisko,AutorAGG.ID_Student,CONCAT(Pracownik.Imie,' ',Pracownik.Nazwisko) AS Promotor,
PracaDyplomowa.OcenaPromotora,PracaDyplomowa.DataZlozenia,Obrona.DataObrony,Obrona.OcenaKoncowa
FROM PracaDyplomowa
LEFT JOIN Obrona ON PracaDyplomowa.ID_Praca = Obrona.ID_Praca
LEFT JOIN Autor ON PracaDyplomowa.ID_Praca = Autor.ID_Praca
Left JOIN Student ON Autor.ID_Student = Student.ID_Student
LEFT JOIN StudiowanyKierunek ON StudiowanyKierunek.ID_Student = Student.ID_Student
LEFT JOIN Studia ON Studia.ID_Studia = StudiowanyKierunek.ID_Studia
LEFT JOIN Pracownik ON PracaDyplomowa.ID_Promotor = Pracownik.ID_Pracownik
LEFT JOIN TypStudiow ON PracaDyplomowa.ID_TypStudiow = TypStudiow.ID_TypStudiow
LEFT JOIN (
    SELECT Autor.ID_Praca AS ID_Praca,STRING_AGG(CONCAT(Student.Imie,' ',Student.Nazwisko),', ') AS ImieNazwisko,STRING_AGG(Student.ID_Student,', ') AS ID_Student
    FROM PracaDyplomowa
    LEFT JOIN Autor ON Autor.ID_Praca = PracaDyplomowa.ID_Praca
    LEFT JOIN Student ON Autor.ID_Student = Student.ID_Student
    GROUP BY Autor.ID_Praca
) AS AutorAGG ON AutorAGG.ID_Praca = PracaDyplomowa.ID_Praca
WHERE Obrona.OcenaKoncowa>=3.0 AND (DATEPART(YEAR,Obrona.DataObrony) = 2021 OR  StudiowanyKierunek.ID_Studia = 1) -- LUB ZAMIAST 'StudiowanyKierunek.ID_Studia = 1' -> (Studia.Kierunek = '' AND Studia.Wydzial = '' )

(4 rows affected)

Total execution time: 00:00:00.092

ID_Praca,TematPracy,Rodzaj,ImieNazwisko,ID_Student,Promotor,OcenaPromotora,DataZlozenia,DataObrony,OcenaKoncowa
1,Grzyby,inzynierskie,"Marcin Bobinski, Dominik Grabiec, Marcel Polak","297225, 297226, 297227",Olga Olszewska,"3,0",2021-01-10 23:06:04.960,NULL,"3,0"
2,Mchy,licencjackie,Magda Nguyen Hong,297228,Olga Olszewska,"3,0",NULL,2021-01-10 21:18:20.110,"4,0"
3,Kora,inzynierskie,Marcin Bobinski,297225,Anna Anielska,"3,0",2021-01-10 23:06:04.977,2020-01-10 21:18:20.110,"5,0"
4,Kotki,licencjackie,Dominik Grabiec,297226,Anna Anielska,"3,0",2021-01-10 23:06:04.980,2020-01-10 21:18:20.110,"4,5"
